In [8]:
using Flux, Statistics
using Flux.Data: DataLoader
using Flux: onehotbatch, onecold, logitcrossentropy, throttle, @epochs
using Base.Iterators: repeated
using Parameters: @with_kw
using CUDAapi
using MLDatasets
if has_cuda()  # Check if CUDA is available
    @info "CUDA is on"
    import CuArrays  # If CUDA is available, import CuArrays
    CuArrays.allowscalar(false)
end

In [21]:
@with_kw mutable struct Args
    η::Float64 = 3e-4       # learning rate
    batchsize::Int = 1024   # batch size
    epochs::Int = 10        # number of epochs
    device::Function = gpu  # set as gpu, if gpu available
end

Args

In [22]:
function getdata(args)
    # Loading Dataset
    xtrain, ytrain = MLDatasets.MNIST.traindata(Float32)
    xtest, ytest = MLDatasets.MNIST.testdata(Float32)

    # Reshape Data for flatten the each image into linear array
    xtrain = Flux.flatten(xtrain)
    xtest = Flux.flatten(xtest)

    # One-hot-encode the labels
    ytrain, ytest = onehotbatch(ytrain, 0:9), onehotbatch(ytest, 0:9)

    # Batching
    train_data = DataLoader(xtrain, ytrain, batchsize=args.batchsize, shuffle=true)
    test_data = DataLoader(xtest, ytest, batchsize=args.batchsize)

    return train_data, test_data
end

getdata (generic function with 1 method)

In [12]:
function build_model(; imgsize=(28,28,1), nclasses=10)
    return Chain(
            Dense(prod(imgsize), 32, relu),
            Dense(32, nclasses))
end

build_model (generic function with 1 method)

In [13]:
function loss_all(dataloader, model)
    l = 0f0
    for (x,y) in dataloader
        l += logitcrossentropy(model(x), y)
    end
    l/length(dataloader)
end

loss_all (generic function with 1 method)

In [16]:
function accuracy(data_loader, model)
    acc = 0
    for (x,y) in data_loader
        acc += sum(onecold(cpu(model(x))) .== onecold(cpu(y)))*1 / size(x,2)
    end
    acc/length(data_loader)
end

accuracy (generic function with 1 method)

In [17]:
function train(; kws...)
    # Initializing Model parameters 
    args = Args(; kws...)

    # Load Data
    train_data,test_data = getdata(args)

    # Construct model
    m = build_model()
    train_data = args.device.(train_data)
    test_data = args.device.(train_data)
    m = args.device(m)
    loss(x,y) = logitcrossentropy(m(x), y)
    
    ## Training
    evalcb = () -> @show(loss_all(train_data, m))
    opt = ADAM(args.η)

    @epochs args.epochs Flux.train!(loss, params(m), train_data, opt, cb = evalcb)

    @show accuracy(train_data, m)

    @show accuracy(test_data, m)
end

train (generic function with 1 method)

In [18]:
cd(@__DIR__)
train()

┌ Info: Epoch 1
└ @ Main /Users/yasu/.julia/packages/Flux/Fj3bt/src/optimise/train.jl:121


loss_all(train_data, m) = 2.2981153f0
loss_all(train_data, m) = 2.2775574f0
loss_all(train_data, m) = 2.257917f0
loss_all(train_data, m) = 2.2389674f0
loss_all(train_data, m) = 2.2206185f0
loss_all(train_data, m) = 2.2027218f0
loss_all(train_data, m) = 2.185217f0
loss_all(train_data, m) = 2.1680458f0
loss_all(train_data, m) = 2.151162f0
loss_all(train_data, m) = 2.1345675f0
loss_all(train_data, m) = 2.118097f0
loss_all(train_data, m) = 2.101754f0
loss_all(train_data, m) = 2.0854404f0
loss_all(train_data, m) = 2.0692234f0
loss_all(train_data, m) = 2.0530176f0
loss_all(train_data, m) = 2.0368364f0
loss_all(train_data, m) = 2.020643f0
loss_all(train_data, m) = 2.0044088f0
loss_all(train_data, m) = 1.988065f0
loss_all(train_data, m) = 1.9716523f0
loss_all(train_data, m) = 1.9551747f0
loss_all(train_data, m) = 1.9386064f0
loss_all(train_data, m) = 1.9219378f0
loss_all(train_data, m) = 1.9051024f0
loss_all(train_data, m) = 1.8881837f0
loss_all(train_data, m) = 1.8712518f0
loss_all(train_data

┌ Info: Epoch 2
└ @ Main /Users/yasu/.julia/packages/Flux/Fj3bt/src/optimise/train.jl:121


1.3210677f0
loss_all(train_data, m) = 1.3079668f0
loss_all(train_data, m) = 1.2950487f0
loss_all(train_data, m) = 1.282315f0
loss_all(train_data, m) = 1.2697116f0
loss_all(train_data, m) = 1.2572827f0
loss_all(train_data, m) = 1.2450247f0
loss_all(train_data, m) = 1.2329206f0
loss_all(train_data, m) = 1.2209444f0
loss_all(train_data, m) = 1.2091535f0
loss_all(train_data, m) = 1.1975375f0
loss_all(train_data, m) = 1.1860894f0
loss_all(train_data, m) = 1.1748438f0
loss_all(train_data, m) = 1.1637418f0
loss_all(train_data, m) = 1.152799f0
loss_all(train_data, m) = 1.142028f0
loss_all(train_data, m) = 1.1314083f0
loss_all(train_data, m) = 1.1209738f0
loss_all(train_data, m) = 1.1107329f0
loss_all(train_data, m) = 1.1006857f0
loss_all(train_data, m) = 1.0907996f0
loss_all(train_data, m) = 1.0811092f0
loss_all(train_data, m) = 1.0715741f0
loss_all(train_data, m) = 1.0622082f0
loss_all(train_data, m) = 1.0530168f0
loss_all(train_data, m) = 1.0439694f0
loss_all(train_data, m) = 1.0350765f0
los

┌ Info: Epoch 3
└ @ Main /Users/yasu/.julia/packages/Flux/Fj3bt/src/optimise/train.jl:121


...

┌ Info: Epoch 9
└ @ Main /Users/yasu/.julia/packages/Flux/Fj3bt/src/optimise/train.jl:121


loss_all(train_data, m) = 0.34758905f0
loss_all(train_data, m) = 0.34718093f0
loss_all(train_data, m) = 0.34680343f0
loss_all(train_data, m) = 0.34645495f0
loss_all(train_data, m) = 0.34612507f0
loss_all(train_data, m) = 0.34581918f0
loss_all(train_data, m) = 0.34549105f0
loss_all(train_data, m) = 0.3451957f0
loss_all(train_data, m) = 0.34491035f0
loss_all(train_data, m) = 0.344577f0
loss_all(train_data, m) = 0.34424326f0
loss_all(train_data, m) = 0.34390843f0
loss_all(train_data, m) = 0.34356788f0
loss_all(train_data, m) = 0.3432545f0
loss_all(train_data, m) = 0.34292975f0
loss_all(train_data, m) = 0.34256428f0
loss_all(train_data, m) = 0.3421744f0
loss_all(train_data, m) = 0.3417915f0
loss_all(train_data, m) = 0.34140337f0
loss_all(train_data, m) = 0.34101942f0
loss_all(train_data, m) = 0.34064895f0
loss_all(train_data, m) = 0.3402781f0
loss_all(train_data, m) = 0.3399288f0
loss_all(train_data, m) = 0.33959642f0
loss_all(train_data, m) = 0.33929637f0
loss_all(train_data, m) = 0.33899

┌ Info: Epoch 10
└ @ Main /Users/yasu/.julia/packages/Flux/Fj3bt/src/optimise/train.jl:121


loss_all(train_data, m) = 0.32891357f0
loss_all(train_data, m) = 0.32857573f0
loss_all(train_data, m) = 0.3282662f0
loss_all(train_data, m) = 0.32798526f0
loss_all(train_data, m) = 0.32772344f0
loss_all(train_data, m) = 0.32748604f0
loss_all(train_data, m) = 0.32722554f0
loss_all(train_data, m) = 0.32699957f0
loss_all(train_data, m) = 0.3267869f0
loss_all(train_data, m) = 0.32652876f0
loss_all(train_data, m) = 0.32626396f0
loss_all(train_data, m) = 0.3259951f0
loss_all(train_data, m) = 0.325717f0
loss_all(train_data, m) = 0.32546526f0
loss_all(train_data, m) = 0.32519805f0
loss_all(train_data, m) = 0.32488668f0
loss_all(train_data, m) = 0.32454887f0
loss_all(train_data, m) = 0.3242214f0
loss_all(train_data, m) = 0.32389003f0
loss_all(train_data, m) = 0.32356456f0
loss_all(train_data, m) = 0.32325408f0
loss_all(train_data, m) = 0.32294258f0
loss_all(train_data, m) = 0.3226528f0
loss_all(train_data, m) = 0.3223817f0
loss_all(train_data, m) = 0.3221424f0
loss_all(train_data, m) = 0.321904

0.9150939451382695